### Consider a problem: $\min cx$ s.t. $Ax \geq b$, $x$ is prime. Assume that its optimal objective function value is $z$. Consider a perturbed problem: $\min (c+c_\delta)x$ s.t. $(A+A_\delta)x \geq b+b_\Delta$, $x$ is prime. Given a nonnegative value $\Delta$. The function below checks whether the objective function value of the perturbed problem is at least $z-\Delta$. 

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import sympy
import time

In [2]:
def furtherFromPrime(x):
    '''
    Find an element of list x which has the highest distance from its closest prime number
    
    Input argument: x (list of numbers)
    
    Output: [i , j] where i is the value of that element and j is its index of the list x
    '''
    j = 0
    max_ = -float('inf')
    furthest = [-1, -1]
    for i in x:
        round_i = int(round(i,0))
        # Check if round(i) is a prime
        if sympy.isprime(round_i):
            distance = abs(i - round_i)
        else:
            next_prime = sympy.nextprime(round_i)
            prev_prime = sympy.prevprime(round_i)
            distance = min(i - prev_prime, next_prime - i)
        if distance > max_:
            max_ = distance
            furthest = [i,j]
        j += 1
    return furthest

def testInteger(x):
    '''
    Test if the given list of number are all integers
    
    Input argument: x (list of numbers)
    
    Output: Boolean, if all elements of x are integer or not
    '''
    test = np.round(x, 6) - np.round(x, 0)
    test = np.abs(test)
    tolerance = np.ones(len(test)) * -1e-6
    if all(test < tolerance) or all(test == 0):
        return True
    else:
        return False

def testPrime(x):
    '''
    Test if the given list of number are all prime numbers
    
    Input argument: x (list of numbers)
    
    Output: Boolean, if all elements of x are prime or not
    '''
    # we have to first test if it is an integer solution
    if testInteger(x):
        test = np.round(x, 6)
        s = 0
        for t in test:
            if not sympy.isprime(int(t)):
                s += 1
        if s==0:
            return True
        else:
            return False
    else:
        return False

def furtherFromInteger(x):
    '''
    Find an element of list of numbers x which is furthest from integer value
    
    Input: x (list of numbers)
    
    Output: [i, j] where i is the value of that element and j is its index 
    '''
    remainder = [np.abs(i - round(i,0)) for i in x]
    j = np.argmax(remainder)
    return [x[j], j] 
    
def Feasibility_Check(dual_variable, b, LB, UB, delta_b, diff_z_p, q_p, delta_q_p):
    '''
    A function to solve an optimization
    Min 1
    s.t. dual_variable[i]*sum([A[i][j]*LB[j]] for j in range(n)) + sum([s_j*(UB[j]-LB[j])] for j in range(n)) - dual_variable[i]*delta_ai <= r_p
         s_j >= 0
         s_j >= -q_j
    is feasible or not

    Parameters
    ----------
    dual_variable : list
    A : list
        LHS of A_i*x >= a_i.
    i : int
        Index of row that the constraint is changed
    LB : list
        Lower bound of the leaf node.
    UB : list
        Upper bound at that leaf node.
    r_p : float
    q_p : list
    delta_ai : float
        Amount of change in the RHS of the constraint A_i*x >= a_i.

    Returns
    -------
    Result : Boolean
            True if there exists such s_j
            False otherwise

    '''
    #Suppressing Gurobi output
    # env = gp.Env(empty=True)
    # env.setParam("OutputFlag",0)
    # env.setParam('InfUnbdInfo', 1)
    # env.start()
    
    # Transform all lists into np.array
    dual_variable = np.array(dual_variable)
    # A = np.array(A)
    LB = np.array(LB)
    UB = np.array(UB)
    q_p = np.array(q_p)
    delta_q_p = np.array(delta_q_p)
    n = len(q_p)
    b = np.array(b)
    delta_b = np.array(delta_b)
    
    RHS = np.array(dual_variable)@(b+delta_b)-diff_z_p
    
    LHS = sum([(q_p[j]+delta_q_p[j])*LB[j] for j in range(n)])
    
    for i in range(n):
        if q_p[i]+delta_q_p[i] > 0:
            LHS += (q_p[i]+delta_q_p[i])*(UB[i]-LB[i])
    
    if LHS <= RHS:
        return True
    else:
        return False

In [3]:
def branch_and_bound_prime_SA(c, A, b, lower_bounds, upper_bounds, Delta=0, delta_A=None, delta_b=None, delta_c=None):
    '''
    Implementation of sensitivity analysis of a Prime Program of the form
    
    z* = min cx
    s.t. Ax >= b
         x in bounds
         x is prime
    
    to the perturbed problem of the form
    
    min (c+delta_c)x
    s.t. (A+delta_A)x >= b+delta_b
         x in bounds
         x is primes
    
    Input: 
        c: list of numbers
        A: list of lists
        b: list of numbers
        lower_bounds: list of numbers which need to manually set to 2 for prime variables if there is no limitation
        upper_bounds: list of numbers
        Delta: nonnegative number
        delta_A: list of lists
        delta_b: list of numbers
        delta_c: list of numbers
    
    Output: 
        SA_status (boolean): True if the objective function value of the perturbed problem is at least z* - Delta, 
        and False if this implication cannot be made through the sensitivity analysis.
    
    '''
    start_time = time.time()
    
    # Transform None matrix to zero matrix
    if delta_A is None:
        delta_A = np.zeros(shape= np.array(A).shape)
    if delta_b is None:
        delta_b = np.zeros(len(b))
    if delta_c is None:
        delta_c = np.zeros(len(c))
        
    # Initialize SA_status = True if the input is allowed for the SA
    SA_status = True
    
    n = len(c)
    epsilon = 1e-10
    # Make the bounds to prime number
    for ii in range(n):
        if not sympy.isprime(upper_bounds[ii]):
            upper_bounds[ii] = sympy.prevprime(upper_bounds[ii])
    
    # Create optimization problem
    #Suppressing Gurobi output
    env = gp.Env(empty=True)
    env.setParam("OutputFlag",0)
    env.setParam('InfUnbdInfo', 1)
    env.start()
    
    # Create a model
    m = gp.Model("Prime Optimization", env = env)
    
    #Variables
    v_name = {}
    x = {}
    for i in range(n):
        v_name[i] = "x{0}".format(i)
        x[i] = m.addVar(lb = lower_bounds[i], ub = upper_bounds[i], name=v_name[i])
        
    m.update()
    
    # add constraints
    for i in range(len(A)):
        m.addConstr(gp.quicksum(A[i][j]*x[j] for j in range(len(A[i]))) >= b[i])
    
    # set objective
    m.setObjective(gp.quicksum(c[i]*x[i] for i in range(len(c))), GRB.MINIMIZE)
    
    # optimize
    m.optimize()
    
    #Check if feasible
    if m.status !=2:
        print("Problem is infeasible!")
        return (0,0,0)
    else:
        # if the root node gives a feasible solution
        if testPrime(m.x):
            return (1, m.objval, m.x)
    
    #Intializing global upper bound and incumbent
    global_ub = 1e12
    incumbent = []
    # print("Initial objective value is ", m.objval)
    
    # Initialize heap
    nodes_model = [m]
    LB = [x[j].LB for j in range(n)]
    UB = [x[j].UB for j in range(n)]
    nodes_LB = [LB]
    nodes_UB = [UB]
    
    #Node count
    node_count = 1
    
    #------------------------------------
    # Starting branch-and-bound iteration
    #------------------------------------ 
    
    while( len(nodes_model) != 0):
        
        #------------------------------------------------------
        #Depth first search: Last node added to the list is choosen
        #-------------------------------------------------------
        m = nodes_model[-1]
        m.optimize()
        # print(m.lb)
        # print(m.ub)
        
        # Check if the node is feasible
        if m.status != 2:
            #Removing parent node
            # print("This node is pruned by infeasibility")
            dual_farkas = m.getAttr('FarkasDual')
            dual_farkas = np.abs(dual_farkas)
            # print(f"Dual Farkas: {dual_farkas}")
            
            # Compute attributes for sensitivity analysis 
            q_p = np.array(dual_farkas)@np.array(A)
            diff_z_p = epsilon
            delta_q_p = np.array(dual_farkas)@np.array(delta_A)
            
            
            # Check if this node satisfy the sufficient condition
            if not Feasibility_Check(dual_variable = dual_farkas, b = b, LB = nodes_LB[-1], UB = nodes_UB[-1], delta_b = delta_b, diff_z_p = diff_z_p, q_p = q_p, delta_q_p = delta_q_p):
                SA_status = False
#                 print("It cannot be implied by the sensitivity analysis.")
                # break
            
            # Removing parent node
            nodes_model.pop(-1)
            nodes_LB.pop(-1)
            nodes_UB.pop(-1)
            
            # Go for the next node
            continue
        
        # If the node is pruned by bound
        elif m.objval >= global_ub:
            # print("This node is pruned by bound")
            
            # Compute attributes for sensitivity analysis 
            Pi = []
            for constr in m.getConstrs():
                Pi.append(abs(constr.Pi))
            q_p = np.array(Pi)@np.array(A)-np.array(c)
            delta_q_p = np.array(Pi)@np.array(delta_A) - np.array(delta_c)
            diff_z_p = global_ub - Delta
            
            
            # Check if this node satisfy the sufficient condition
            if not Feasibility_Check(dual_variable = Pi, b = b, LB = nodes_LB[-1], UB = nodes_UB[-1], delta_b = delta_b, diff_z_p = diff_z_p, q_p = q_p, delta_q_p = delta_q_p):
                SA_status = False
#                 print("It cannot be implied by the sensitivity analysis.")
                # break
            
            # Removing parent node
            nodes_model.pop(-1)
            nodes_LB.pop(-1)
            nodes_UB.pop(-1)
            
            # Go for the next node
            continue
        
        # The node is further branched on
        else:
            # print("Relaxation solution of the current node is ", m.x, m.status)
            
            # Compute attributes for sensitivity analysis 
            Pi = []
            for constr in m.getConstrs():
                Pi.append(abs(constr.Pi))
            q_p = np.array(Pi)@np.array(A)-np.array(c)
            delta_q_p = np.array(Pi)@np.array(delta_A) - np.array(delta_c)
            diff_z_p = m.objval - Delta
            
            # update the global upper bound
            if m.objval < global_ub and testPrime(m.x):
                global_ub = m.objval
                # print("global upper bound is updated to ", global_ub)
                incumbent = m.x
                # prune by optimality
                # print("This node is pruned by optimality")
                
                # Check if this node satisfy the sufficient condition
                if not Feasibility_Check(dual_variable = Pi, b = b, LB = nodes_LB[-1], UB = nodes_UB[-1], delta_b = delta_b, diff_z_p = diff_z_p, q_p = q_p, delta_q_p = delta_q_p):
                    SA_status = False
#                     print("It cannot be implied by the sensitivity analysis.")
                    # break
                
                # Removing parent node
                nodes_model.pop(-1)
                nodes_LB.pop(-1)
                nodes_UB.pop(-1)
                
                # Go for the next node
                continue
            else: # the node is further explored
                # find which variables to be branched
                # find variable that is furthest from prime
                branch_ind = int(furtherFromPrime(m.x)[1])
                # print('Branch index = ', branch_ind)
            
                # subproblem 1 branch down
                left_model = m.copy()
                branch_val = sympy.prevprime(m.x[branch_ind])
                # print('Branch value previous prime = ', branch_val)
        
                # Generate variable
                x = {}
                for i in range(n):
                    x[i] = left_model.getVarByName(v_name[i])
            
                x[branch_ind].setAttr("UB", branch_val)
                left_model.update()
                
                # retrieve LB and UB
                left_LB = [x[j].LB for j in range(n)]
                left_UB = [x[j].UB for j in range(n)]
        
                # subproblem 2 branch up
                right_model = m.copy()
                branch_val = sympy.nextprime(m.x[branch_ind])
                # print('Branch value next prime = ', branch_val)
        
                # create variables
                x = {}
                for i in range(n):
                    x[i] = right_model.getVarByName(v_name[i])
            
                x[branch_ind].setAttr("LB", branch_val)
                right_model.update()
                
                # retrieve LB and UB
                right_LB = [x[j].LB for j in range(n)]
                right_UB = [x[j].UB for j in range(n)]
                
                #Removing parent node
                nodes_model.pop(-1)
                nodes_LB.pop(-1)
                nodes_UB.pop(-1)
        
                # Store the node      
                nodes_model.append(right_model)
                nodes_model.append(left_model)
                # Store LB
                nodes_LB.append(right_LB)
                nodes_LB.append(left_LB)
                # Store UB
                nodes_UB.append(right_UB)
                nodes_UB.append(left_UB)
        
                node_count += 2
            
    end_time = time.time()
    computation_time = end_time - start_time
    
    return SA_status

### Input matrices here

In [4]:
c = np.array([-1, 0])
A = np.array([[1, 1], [-1, -1]])
b = np.array([1000, -1000])
lower_bounds=np.array([2,2])
upper_bounds = np.array([100000,100000])

Delta = 0
delta_A = np.array([[0,0],[0,0]])
delta_c = np.array([0,0])
delta_b = np.array([4,-4])

In [5]:
SA_status = branch_and_bound_prime_SA(c=c, A=A, b=b, lower_bounds = lower_bounds, upper_bounds = upper_bounds,\
                                      Delta= Delta, delta_A=delta_A, delta_b=delta_b, delta_c=delta_c)
print(SA_status)

True
